In [22]:
import nest_asyncio
nest_asyncio.apply()

from sc2 import maps
from sc2.bot_ai import BotAI
from sc2.data import Difficulty, Race
from sc2.ids.ability_id import AbilityId
from sc2.ids.buff_id import BuffId
from sc2.ids.unit_typeid import UnitTypeId
from sc2.main import run_game
from sc2.player import Bot, Computer
import random
import math
import numpy as np; from numpy import linspace, pi

### needs to be a state machine

class BBBot(BotAI):
    # pylint: disable=R0912
    async def on_step(self, iteration):
        if iteration % 5 == 0:
            self.tick5 = True
        else:
            self.tick5 = False
        
        bases = self.townhalls
        num_bases = self.townhalls.amount
        num_workers = self.workers.amount
        if num_bases > 0:
            worker_ratio = num_workers/num_bases
        else:
            worker_ratio = 0

        await self.distribute_workers()
        #await self.build_supply()
        await self.build_pylons()
        #await self.build_marines()
        #await self.marine_patrol2()
        #await self.status()

        if worker_ratio < 15:
            await self.build_workers()
        else:
            await self.expand()

    async def build_workers(self):
        for th in self.townhalls.ready.idle:
            if self.can_afford(UnitTypeId.PROBE):
                th.train(UnitTypeId.PROBE)

    async def expand(self):
        if self.townhalls.amount < 3 and self.can_afford(UnitTypeId.NEXUS):
            await self.expand_now()
        #else:
            #for th in self.townhalls:

    async def build_pylons(self):
        ths = self.townhalls.ready.idle
        if ths.__len__() > 1:
            for th in ths:
                if self.structures(UnitTypeId.PYLON).closer_than(20, th).amount < 6 and self.already_pending(UnitTypeId.PYLON) < 6:
                    if self.can_afford(UnitTypeId.PYLON):
                        goal = self.structures(UnitTypeId.NEXUS).closest_to(th)
                        nearloc = th.position.towards(goal)
                        await self.build(UnitTypeId.PYLON, near=nearloc, max_distance=50)


def main():
    run_game(
        maps.get("(2)CatalystLE"),
        [Bot(Race.Protoss, BBBot()),
         Computer(Race.Terran, Difficulty.Hard)],
        realtime=False,
    )


if __name__ == "__main__":
    main()


2023-02-05 18:55:19.613 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.launched (was None)
2023-02-05 18:55:19.614 | INFO     | sc2.controller:create_game:37 - Creating new game
2023-02-05 18:55:19.614 | INFO     | sc2.controller:create_game:38 - Map:     (2)CatalystLE
2023-02-05 18:55:19.614 | INFO     | sc2.controller:create_game:39 - Players: Bot BBBot(Protoss), Computer Hard(Terran, RandomBuild)
2023-02-05 18:55:19.616 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.init_game (was Status.launched)
2023-02-05 18:55:24.792 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.in_game (was None)
2023-02-05 18:55:24.793 | INFO     | sc2.main:_play_game:221 - Player 1 - Bot BBBot(Protoss)
2023-02-05 18:55:59.230 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.ended (was Status.in_game)
2023-02-05 18:55:59.231 | INFO     | sc2.main:_play_game:228 - Result for player 1 - Bot BBBot(Protoss): Defeat
2